In [1]:
import os
import numpy as np
import gzip
import pickle

import torch
from torch_geometric.data import Dataset, Data, DataLoader

import import_ipynb
from influenceDataset import get_dataloader_10000_noprocessing
from gnnNets import get_gnnNets

importing Jupyter notebook from influenceDataset.ipynb
importing Jupyter notebook from gnnNets.ipynb


In [8]:
GPU_NUM = 0

device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

print(device)

cuda:0


In [56]:
lr = 1e-3
epochs = 1000
batch_size = 20

In [5]:
edge_index = [[],[]]
edge_attr = []
with open('/data/URP/graphs/Celebrity_train_JI.txt','r') as f:
    n,m = map(int,f.readline().split())
    for _ in range(m):
        u,v,p = f.readline().split()
        u,v,p = int(u),int(v),float(p)
        edge_index[0].append(u)
        edge_index[1].append(v)
        edge_attr.append([p])
edge_index = torch.tensor(edge_index)
edge_attr = torch.tensor(edge_attr)

with gzip.open('/data/URP/raw/Celebrity_train_JI_350.pkl.gz', 'rb') as f: seeds,probs = pickle.load(f)

In [10]:
seeds = torch.from_numpy(np.eye(2)[seeds.astype(int)]).float()
probs = torch.from_numpy(np.sum(probs,axis=-1,keepdims=True)).float()

datas = []
for i in range(len(seeds)):
    seed,prob = seeds[i],probs[i]
    prob = torch.unsqueeze(prob, 0)  # 이렇게 해야 loss를 계산할 때 자연스럽다.
    data = Data(x=seed, edge_index=edge_index, edge_attr=edge_attr, y=prob)
    datas.append(data)



train_num = int(len(datas)*0.8)
val_num = int(len(datas)*0.1)
test_num = len(datas)-train_num-val_num

train_dataloader = DataLoader(datas[:train_num], batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(datas[train_num:train_num+val_num], batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(datas[train_num+val_num:], batch_size=batch_size, shuffle=False)

/opt/conda/envs/URP/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [57]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [53]:
model = get_gnnNets(2, 1, {'gnn_latent_dim':[128,128,128]})
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss(reduction='sum')

best_val_loss = float('inf')
best_model_state_dict = None

In [58]:
# torch.save 주석처리해둠.

for epoch in range(epochs):
    if epoch%1==0 : print(f'----- epoch {epoch+1} / {epochs} -----')
    # train
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        batch = batch.to(device)
        
        preds = model(batch)
        
        loss = loss_fn(preds,batch.y)
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    train_loss /= len(train_dataloader.dataset)
    
    """
    # validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch in val_dataloader:
            batch = batch.to(device)
            
            preds = model(batch)
            
            loss = loss_fn(preds,batch.y)
            val_loss += loss.item()
        val_loss /= len(val_dataloader.dataset)
    
    if best_val_loss > val_loss:
        best_val_loss = val_loss
        best_model_state_dict = model.state_dict()
    """
    
    if epoch%1==0 :
        print('train loss:',train_loss**0.5)
        #print('val loss:',val_loss**0.5)
print('training end\n')
    
model.load_state_dict(best_model_state_dict)
#torch.save(best_model_state_dict, "/data/URP/model/"+'bestmodel.pt')

        
# test
model.eval()
with torch.no_grad():
    test_loss = 0
    for batch in test_dataloader:
        batch = batch.to(device)
        
        preds = model(batch)
        
        loss = loss_fn(preds,batch.y)
        test_loss += loss.item()
    test_loss /= len(test_dataloader.dataset)
print('test loss:', test_loss**0.5)

----- epoch 1 / 1000 -----
train loss: 873.4813842157305
----- epoch 2 / 1000 -----
train loss: 868.1574248289962
----- epoch 3 / 1000 -----
train loss: 822.9960651268545
----- epoch 4 / 1000 -----
train loss: 582.1147315729827
----- epoch 5 / 1000 -----
train loss: 468.63787706653477
----- epoch 6 / 1000 -----
train loss: 440.3991827632874
----- epoch 7 / 1000 -----
train loss: 441.5652485760173
----- epoch 8 / 1000 -----
train loss: 444.9876483036997
----- epoch 9 / 1000 -----
train loss: 437.10541798125934
----- epoch 10 / 1000 -----
train loss: 434.7858596481721
----- epoch 11 / 1000 -----
train loss: 431.92406636716004
----- epoch 12 / 1000 -----
train loss: 434.2012021270455
----- epoch 13 / 1000 -----
train loss: 434.4986961594772
----- epoch 14 / 1000 -----
train loss: 432.85967084363165
----- epoch 15 / 1000 -----
train loss: 433.04637556831847
----- epoch 16 / 1000 -----
train loss: 433.17203362438545
----- epoch 17 / 1000 -----
train loss: 432.6246868162485
----- epoch 18 / 

KeyboardInterrupt: 

In [ ]:
# 100 epoch에 30분정도
# 한 250epoch쯤 되면 loss 450정도에서 멈춤

In [52]:
model.eval()
with torch.no_grad():
    test_loss = 0
    for batch in test_dataloader:
        batch = batch.to(device)
        
        preds = model(batch)
        
        loss = loss_fn(preds,batch.y)
        print(preds,batch.y)
        test_loss += loss.item()
    test_loss /= len(test_dataloader.dataset)
print('test loss:', test_loss**0.5)

tensor([[771.6221],
        [750.9333],
        [780.3817],
        [779.9252],
        [771.3115],
        [776.5215],
        [749.6232],
        [765.3770],
        [781.3361],
        [776.5773],
        [751.1804],
        [783.3389],
        [753.1251],
        [750.7109],
        [754.6161],
        [779.3143],
        [778.4957],
        [779.5428],
        [771.3115],
        [751.1580]], device='cuda:0') tensor([[ 824.3052],
        [ 523.8202],
        [1084.5341],
        [1439.6285],
        [1463.5371],
        [ 181.8262],
        [  12.8440],
        [ 846.4346],
        [1214.1984],
        [ 293.4373],
        [ 452.6539],
        [1374.0746],
        [ 972.0948],
        [ 296.7485],
        [1085.3948],
        [1465.4924],
        [ 698.3870],
        [1167.0297],
        [  89.1388],
        [ 636.2567]], device='cuda:0')
tensor([[754.6353],
        [752.3193],
        [771.3115],
        [781.1602],
        [754.5455],
        [765.3602],
        [750.5134],
    